In [17]:
import pandas as pd
from dash import Dash, html, dcc, callback, Output, Input
import censusgeocode as cg
import plotly.figure_factory as ff
from urllib.request import urlopen
import json
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
from sklearn.linear_model import LinearRegression
from Partisian import get_state_part_score
from Life import get_age_life_fig, get_state_age_life_exp
from sklearn.neighbors import KNeighborsClassifier
from plotly.subplots import make_subplots
import math
from LifeData import LifeData
import Partisian
from sklearn.linear_model import LinearRegression
import addfips


,Notes,State,State Code,Deaths,Population,Crude Rate
0,NaN,Alabama,1.0,54108.0,4903185.0,1103.5
1,NaN,Alaska,2.0,4613.0,731545.0,630.6
2,NaN,Arizona,4.0,60236.0,7278717.0,827.6
3,NaN,Arkansas,5.0,32888.0,3017804.0,1089.8
4,NaN,California,6.0,269831.0,39512223.0,682.9
...,...,...,...,...,...,...
90,estimates. Population figures for the infant a...,NaN,NaN,NaN,NaN,NaN
91,figures for years 2001 - 2009 differ slightly ...,NaN,NaN,NaN,NaN,NaN
92,were available at the time of release.,NaN,NaN,NaN,NaN,NaN
93,2. The population figures used in the calculat...,NaN,NaN,NaN,NaN,NaN


In [149]:
def translate_ICD10(x, conversion):
    try:
        return conversion[x]
    except:
        return x
    
    



def clean_state_data(df):
    af = addfips.AddFIPS()
    try:
        df = df.dropna(subset=["State","ICD-10 113 Cause List Code", "Population"])
    except: 
        df = df.dropna(subset=["State", "Population"])
    #df = df[df["Crude Rate"] != "Unreliable"]
    df.Population = df.Population.astype(float)
    df["Rate"] = (df["Deaths"]/df["Population"])*100000
    df["Rate"] = df["Rate"].round(2)
    df["fips"] = df.apply(lambda x: af.get_state_fips(x["State"]), axis=1)
    df["Year"] = 2019
    df = df.sort_values(by=["Rate"], ascending=False)
    df.drop(columns=["Notes", "State Code", ], inplace=True)
   
    try:
        f = open("data/Life/Deaths/ICD10Translation.json")
        translation = json.load(f)
        df["ICD-10 113 Cause List"] = df["ICD-10 113 Cause List"].apply(lambda x: translate_ICD10(x, translation))
        df.rename(columns={"ICD-10 113 Cause List":"Cause Of Death"}, inplace=True)
    except:
        None
    return df
    


def top_n(df, n):
    #Filter out low occuring chapters
    df = df.groupby("ICD-10 113 Cause List Code").filter(lambda x: len(x)>1)
    df = df.sort_values(by = ["Rate"], ascending = False)
    df = df.head(n).copy()
    top_codes = list(df["ICD-10 113 Cause List Code"].unique())
    df = df[df["ICD-10 113 Cause List Code"].isin(top_codes)]
    return df


firearms = pd.read_csv(r"data/Life/Deaths/FireArmState.txt", delimiter="	", na_values = ['Not Applicable'])
df = pd.read_csv(r"data/Life/Deaths/StateDeathsAge.txt", delimiter="	", na_values = ['Not Applicable'])
state_pop = pd.read_csv(r"data/Life/Deaths/State_Pop.txt", delimiter="	", na_values = ['Not Applicable'])
yearly_pop = pd.read_csv(r"data/Life/Deaths/2019State1YearPop.txt", delimiter="	", na_values = ['Not Applicable'])


df = clean_state_data(df)
firearms = clean_state_data(firearms)
state_pop = clean_state_data(state_pop)
yearly_pop = clean_state_data(yearly_pop)
state_pop["Total Pop"] = state_pop["Population"]

#top_df = top_n(df, 5)
#county = df.groupby(by=["2006 Urbanization Code", "ICD Sub-Chapter"]).mean().reset_index().sort_values(by=["Rate"], ascending=False)
state_pop.sample(5)
firearms = firearms.merge(state_pop[["State", "Total Pop"]], left_on="State",right_on="State")

In [177]:
df["age"] = df["Ten-Year Age Groups Code"].str.replace("+","").str.split("-").str[0].astype(int)
df

C:\Users\zeusg\AppData\Local\Temp/ipykernel_2808/3922431193.py:1: FutureWarning:

The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.



,State,Cause Of Death,ICD-10 113 Cause List Code,Ten-Year Age Groups,Ten-Year Age Groups Code,Deaths,Population,Crude Rate,Rate,fips,Year,age
10393,Michigan,Heart Disease (Major),GR113-053,85+ years,85+,13436.0,213136.0,6304.0,6303.96,26,2019,85
16288,Ohio,Heart Disease (Major),GR113-053,85+ years,85+,14910.0,256272.0,5818.0,5818.04,39,2019,85
182,Alabama,Heart Disease (Major),GR113-053,85+ years,85+,5294.0,91543.0,5783.1,5783.07,01,2019,85
20777,Utah,Heart Disease (Major),GR113-053,85+ years,85+,2196.0,38574.0,5693.0,5692.95,49,2019,85
1458,Arkansas,Heart Disease (Major),GR113-053,85+ years,85+,3409.0,59912.0,5690.0,5690.01,05,2019,85
...,...,...,...,...,...,...,...,...,...,...,...,...
1924,California,Hodgkin disease (C81),GR113-038,35-44 years,35-44,10.0,5282100.0,Unreliable,0.19,06,2019,35
2197,California,Chronic lower respiratory diseases (All),GR113-082,15-24 years,15-24,10.0,5180973.0,Unreliable,0.19,06,2019,15
1745,California,"Certain other intestinal infections (A04,A07-A09)",GR113-003,35-44 years,35-44,10.0,5282100.0,Unreliable,0.19,06,2019,35
1873,California,Malignant neoplasms of corpus uteri and uterus...,GR113-031,25-34 years,25-34,11.0,6052952.0,Unreliable,0.18,06,2019,25


In [95]:
firearms.sample(3)

,State,Single-Year Ages,Single-Year Ages Code,Deaths,Population,Crude Rate,Rate,fips,Year,Total Pop
2418,West Virginia,27 years,27,42.0,485570.0,8.6,8.65,54,2019,1792147.0
2741,Massachusetts,NaN,NaN,3524.0,144921428.0,2.4,2.43,25,2019,6892503.0
954,North Carolina,28 years,28,392.0,2786483.0,14.1,14.07,37,2019,10488084.0


In [152]:
yearly_pop.Population.sum()

307147383.0

In [19]:
df["Ten-Year Age Groups"].unique()  

array(['85+ years', '75-84 years', '65-74 years', '55-64 years',
       '< 1 year', '45-54 years', '35-44 years', '25-34 years',
       '15-24 years', '1-4 years', '5-14 years'], dtype=object)

In [20]:
teen = df[df["Ten-Year Age Groups"] == "15-24 years"]
teen.sample(3)

,State,Cause Of Death,ICD-10 113 Cause List Code,Ten-Year Age Groups,Ten-Year Age Groups Code,Deaths,Population,Crude Rate,Rate,fips,Year
7836,Kansas,Homicide (All),GR113-127,15-24 years,15-24,32.0,412813.0,7.8,7.75,20,2019
19752,Tennessee,Accidents (All),GR113-112,15-24 years,15-24,349.0,869205.0,40.2,40.15,47,2019
15154,New York,Accidental poisoning,GR113-122,15-24 years,15-24,193.0,2445246.0,7.9,7.89,36,2019


In [21]:
teen_gun = teen[teen["Cause Of Death"] == "Homicide Guns"]
guns = df[df["Cause Of Death"] == "Homicide Guns"]


In [22]:
fig  = go.Figure()
fig.add_trace(go.Bar(x = teen_gun["State"], y = teen_gun["Rate"]))

In [23]:
guns = guns.sort_values(by="Rate", ascending=False)
px.bar(guns, x = "State", y="Rate", color="Ten-Year Age Groups")

In [24]:
age_guns = guns.groupby(["Ten-Year Age Groups"]).mean().reset_index()
age_guns["Start Age"] = age_guns["Ten-Year Age Groups"].str.split("-").str[0].astype(int)
age_guns = age_guns.sort_values("Start Age")
age_guns

C:\Users\zeusg\AppData\Local\Temp/ipykernel_2808/2145025576.py:1: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.



,Ten-Year Age Groups,Deaths,Population,Rate,Year,Start Age
4,5-14 years,13.500000,2.555442e+06,0.700000,2019.0,5
0,15-24 years,105.292683,1.006978e+06,12.293659,2019.0,15
1,25-34 years,111.450000,1.106145e+06,11.635250,2019.0,25
2,35-44 years,66.948718,1.025343e+06,7.672308,2019.0,35
3,45-54 years,42.266667,1.183749e+06,4.421667,2019.0,45
5,55-64 years,29.400000,1.261538e+06,2.721600,2019.0,55
6,65-74 years,16.083333,1.266419e+06,1.572500,2019.0,65
7,75-84 years,13.500000,1.391170e+06,1.010000,2019.0,75


In [25]:
fig = go.Figure()
fig.add_trace(go.Pie(labels=age_guns["Ten-Year Age Groups"], values=age_guns["Rate"], name="Guns"))


In [26]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=age_guns["Ten-Year Age Groups"], y=age_guns["Rate"], mode ="lines" ))

In [27]:
guns
guns["Start Age"] = guns["Ten-Year Age Groups"].str.split("-").str[0].astype(int)
guns = guns.sort_values("Start Age")
guns.sample(3)

,State,Cause Of Death,ICD-10 113 Cause List Code,Ten-Year Age Groups,Ten-Year Age Groups Code,Deaths,Population,Crude Rate,Rate,fips,Year,Start Age
2484,California,Homicide Guns,GR113-128,75-84 years,75-84,14.0,1701599.0,Unreliable,0.82,06,2019,75
1733,Arkansas,Homicide Guns,GR113-128,55-64 years,55-64,19.0,385994.0,Unreliable,4.92,05,2019,55
12250,Missouri,Homicide Guns,GR113-128,45-54 years,45-54,57.0,732944.0,7.8,7.78,29,2019,45


In [28]:
fig = px.line(x = guns["Ten-Year Age Groups"], y= guns["Rate"], color = guns["State"])
fig

In [29]:
firearms.sample(3)

,State,Single-Year Ages,Single-Year Ages Code,Deaths,Population,Crude Rate,Rate,fips,Year
3048,Virginia,84 years,84,11.0,475791.0,Unreliable,2.31,51,2019
2749,Tennessee,13 years,13,21.0,1836570.0,1.1,1.14,47,2019
2143,New York,57 years,57,101.0,5332013.0,1.9,1.89,36,2019


In [30]:
age_guns.sample(3)

,Ten-Year Age Groups,Deaths,Population,Rate,Year,Start Age
6,65-74 years,16.083333,1.266419e+06,1.572500,2019.0,65
3,45-54 years,42.266667,1.183749e+06,4.421667,2019.0,45
7,75-84 years,13.500000,1.391170e+06,1.010000,2019.0,75


In [31]:
firearms

,State,Single-Year Ages,Single-Year Ages Code,Deaths,Population,Crude Rate,Rate,fips,Year
551,District of Columbia,17 years,17,100.0,124053.0,80.6,80.61,11,2019
556,District of Columbia,22 years,22,172.0,226128.0,76.1,76.06,11,2019
555,District of Columbia,21 years,21,157.0,228388.0,68.7,68.74,11,2019
552,District of Columbia,18 years,18,137.0,201938.0,67.8,67.84,11,2019
554,District of Columbia,20 years,20,163.0,241633.0,67.5,67.46,11,2019
...,...,...,...,...,...,...,...,...,...
1946,New Jersey,9 years,9,10.0,2524672.0,Unreliable,0.40,34,2019
301,California,10 years,10,43.0,11451816.0,0.4,0.38,06,2019
298,California,7 years,7,40.0,11120175.0,0.4,0.36,06,2019
2097,New York,11 years,11,16.0,5392660.0,Unreliable,0.30,36,2019


In [98]:
firearms.rename(columns={"Single-Year Ages Code":"age"}, inplace=True)
firearms.dropna(subset=['age'], inplace=True)
firearms["age"] = firearms["age"].astype(int)
firearms = firearms.sort_values(by=["age","Rate"])


firearms_total = firearms.groupby(["age"]).mean().reset_index()

firearms_total.sample(10)

C:\Users\zeusg\AppData\Local\Temp/ipykernel_2808/537691258.py:7: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.



,age,Deaths,Population,Rate,Year,Total Pop
30,30,207.395833,1.909171e+06,10.715000,2019.0,6.781120e+06
57,57,70.743590,2.038843e+06,3.889231,2019.0,8.062741e+06
12,12,21.869565,3.090328e+06,0.841739,2019.0,1.099075e+07
17,17,194.317073,2.192639e+06,9.991220,2019.0,7.722494e+06
73,73,26.233333,1.223940e+06,2.473333,2019.0,9.358104e+06
7,7,21.050000,3.284256e+06,0.735500,2019.0,1.217296e+07
46,46,112.760870,2.026053e+06,6.070000,2019.0,7.044475e+06
52,52,90.095238,2.123903e+06,4.592619,2019.0,7.617178e+06
69,69,32.838710,1.471179e+06,2.417742,2019.0,9.308909e+06
6,6,20.695652,2.986924e+06,0.829565,2019.0,1.114399e+07


In [99]:
firearms

,State,Single-Year Ages,age,Deaths,Population,Crude Rate,Rate,fips,Year,Total Pop
2738,Massachusetts,< 1 year,0,42.0,1630569.0,2.6,2.58,25,2019,6892503.0
3167,New Hampshire,< 1 year,0,13.0,296047.0,Unreliable,4.39,33,2019,1359711.0
1375,California,< 1 year,0,541.0,11038886.0,4.9,4.90,06,2019,39512223.0
938,Delaware,< 1 year,0,12.0,240973.0,Unreliable,4.98,10,2019,973764.0
1880,New Jersey,< 1 year,0,117.0,2343426.0,5.0,4.99,34,2019,8882190.0
...,...,...,...,...,...,...,...,...,...,...
2076,Texas,84 years,84,38.0,1258272.0,3.0,3.02,48,2019,28995881.0
1658,Indiana,84 years,84,15.0,440753.0,Unreliable,3.40,18,2019,6732219.0
875,Georgia,84 years,84,19.0,472598.0,Unreliable,4.02,13,2019,10617423.0
262,Mississippi,84 years,84,11.0,184405.0,Unreliable,5.97,28,2019,2976149.0


In [34]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=firearms_total["age"], y=firearms_total["Rate"], mode ="lines" ))

In [35]:
px.bar(firearms, x = "State", y="Rate", color="age", color_continuous_scale=px.colors.sequential.Bluered, title="Firearm Death Rate by Age Group", labels={"Rate":"Firearm Death Rate (per 100,000)"}, height=600)

In [131]:
guns.sort_values(by="Rate", ascending=False, inplace=True)
px.bar(guns, x = "State", y="Rate", color="Ten-Year Age Groups Code", color_continuous_scale=px.colors.sequential.Bluered, title="Firearm Death Rate by Age Group", labels={"Rate":"Firearm Death Rate (per 100,000)"}, height=600)

In [129]:
guns

,State,Cause Of Death,ICD-10 113 Cause List Code,Ten-Year Age Groups,Ten-Year Age Groups Code,Deaths,Population,Crude Rate,Rate,fips,Year,Start Age
2477,California,Homicide Guns,GR113-128,5-14 years,5-14,15.0,5007987.0,Unreliable,0.30,06,2019,5
20591,Texas,Homicide Guns,GR113-128,5-14 years,5-14,20.0,4166702.0,0.5,0.48,48,2019,5
16668,Ohio,Homicide Guns,GR113-128,5-14 years,5-14,12.0,1440279.0,Unreliable,0.83,39,2019,5
6427,Illinois,Homicide Guns,GR113-128,5-14 years,5-14,13.0,1575745.0,Unreliable,0.83,17,2019,5
12246,Missouri,Homicide Guns,GR113-128,5-14 years,5-14,10.0,769540.0,Unreliable,1.30,29,2019,5
...,...,...,...,...,...,...,...,...,...,...,...,...
22301,Washington,Homicide Guns,GR113-128,65-74 years,65-74,10.0,735524.0,Unreliable,1.36,53,2019,65
19856,Tennessee,Homicide Guns,GR113-128,65-74 years,65-74,11.0,681396.0,Unreliable,1.61,47,2019,65
513,Alabama,Homicide Guns,GR113-128,65-74 years,65-74,17.0,501447.0,Unreliable,3.39,01,2019,65
20598,Texas,Homicide Guns,GR113-128,75-84 years,75-84,13.0,1080742.0,Unreliable,1.20,48,2019,75


In [101]:
part = Partisian.get_state_part_score()
final = pd.merge(part, firearms, on="fips")
final[final["State_x"] == "VT"]

,State_x,fips,Part_Score,Year_x,category,State_y,Single-Year Ages,age,Deaths,Population,Crude Rate,Rate,Year_y,Total Pop
888,VT,50,-0.4,2024,Left,Vermont,< 1 year,0,12.0,133915.0,Unreliable,8.96,2019,623989.0


In [112]:
state_totals = final.groupby(by=["fips","State_y"]).mean().reset_index().sort_values(by="Rate")
state_totals["Rate"] = (state_totals["Deaths"]/state_totals["Total Pop"])*100000

C:\Users\zeusg\AppData\Local\Temp/ipykernel_2808/1594983157.py:1: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.



In [118]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=state_totals["Part_Score"], y=state_totals["Rate"], mode = "markers",  hovertext=state_totals["State_y"], marker=dict(size=state_totals["Total Pop"]/1000000, colorscale='bluered', color=state_totals["Part_Score"])))

In [124]:
state_totals.sort_values("Rate", ascending=False, inplace=True)
fig = go.Figure()
fig.add_trace(go.Bar(x=state_totals["State_y"], y=state_totals["Rate"], name="Rate"'))
fig.show()

SyntaxError: EOL while scanning string literal (Temp/ipykernel_2808/1694211578.py, line 3)

In [115]:
state_totals[state_totals["State_y"] == "Vermont"]

,fips,State_y,Part_Score,Year_x,age,Deaths,Population,Rate,Year_y,Total Pop
43,50,Vermont,-0.4,2024.0,0.0,12.0,133915.0,1.923111,2019.0,623989.0


In [173]:
state_totals

,fips,State_y,Part_Score,Year_x,age,Deaths,Population,Rate,Year_y,Total Pop
17,22,Louisiana,0.333333,2024.0,41.879518,154.987952,1.168366e+06,3.333939,2019.0,4648794.0
23,28,Mississippi,0.800000,2024.0,43.217949,91.115385,7.501262e+05,3.061520,2019.0,2976149.0
7,10,Delaware,-1.000000,2024.0,31.914286,26.714286,2.608420e+05,2.743405,2019.0,973764.0
1,02,Alaska,0.466667,2024.0,35.045455,19.636364,2.202748e+05,2.684232,2019.0,731545.0
0,01,Alabama,0.866667,2024.0,41.835443,126.860759,1.225204e+06,2.587313,2019.0,4903185.0
29,35,New Mexico,-0.466667,2024.0,39.875000,53.031250,5.491884e+05,2.529117,2019.0,2096829.0
19,24,Maryland,-0.533333,2024.0,41.746667,149.053333,1.515354e+06,2.465452,2019.0,6045680.0
48,56,Wyoming,0.866667,2024.0,26.222222,12.777778,1.617492e+05,2.207789,2019.0,578759.0
3,05,Arkansas,0.333333,2024.0,42.426667,64.906667,7.459389e+05,2.150791,2019.0,3017804.0
38,45,South Carolina,1.000000,2024.0,41.777778,102.679012,1.183876e+06,1.994265,2019.0,5148714.0


In [79]:
12/133915.0

8.960908038681252e-05

In [117]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=state_totals["Population"], y=state_totals["Rate"], mode = "markers", hovertext=state_totals["State_y"], marker=dict(colorscale='bluered', color=state_totals["Population"])))

In [125]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=state_totals["age"], y=state_totals["Rate"], mode = "markers", hovertext=state_totals["State_y"], marker=dict(colorscale='bluered', color=state_totals["Population"])))

In [144]:
teen_deaths = teen.groupby(by=["Cause Of Death"]).sum().reset_index()
teen_deaths.sort_values(by=["Deaths"], ascending=False, inplace=True)   

C:\Users\zeusg\AppData\Local\Temp/ipykernel_2808/320830507.py:1: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.



In [145]:
fig = go.Figure()
fig.add_trace(go.Bar(x=teen_deaths["Cause Of Death"], y=teen_deaths["Deaths"], name="Deaths"))

In [153]:
yearly_pop

,State,Single-Year Ages,Single-Year Ages Code,Deaths,Population,Crude Rate,Rate,fips,Year
1638,Kentucky,84 years,84,1256.0,14110.0,8901.5,8901.49,21,2019
4487,West Virginia,84 years,84,649.0,7303.0,8886.8,8886.76,54,2019
3389,Oklahoma,84 years,84,1092.0,12946.0,8435.0,8435.04,40,2019
167,Alaska,84 years,84,106.0,1277.0,8300.7,8300.70,02,2019
3939,Tennessee,84 years,84,1787.0,21687.0,8240.0,8239.96,47,2019
...,...,...,...,...,...,...,...,...,...
382,California,9 years,9,38.0,493445.0,7.7,7.70,06,2019
1191,Illinois,8 years,8,11.0,155612.0,Unreliable,7.07,17,2019
3214,Ohio,5 years,5,10.0,142314.0,Unreliable,7.03,39,2019
1190,Illinois,6 years,6,10.0,151861.0,Unreliable,6.58,17,2019


In [170]:
usa_total = yearly_pop.groupby("Single-Year Ages").sum().reset_index()
usa_total["age"] = usa_total["Single-Year Ages"].str.replace("< 1 year","0").str.split(" ").str[0].astype(int)
usa_total.sort_values("age", inplace=True)
usa_total["Rate"] = (usa_total["Deaths"]/usa_total["Population"])*100000
usa_total

C:\Users\zeusg\AppData\Local\Temp/ipykernel_2808/196067746.py:1: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.



,Single-Year Ages,Deaths,Population,Rate,Year,age
84,< 1 year,20921.0,3783052.0,553.019097,102969,0
0,1 year,1401.0,3621711.0,38.683374,74703,1
11,2 years,835.0,3463061.0,24.111617,62589,2
22,3 years,596.0,3228814.0,18.458790,48456,3
33,4 years,488.0,3201427.0,15.243202,48456,4
...,...,...,...,...,...,...
78,80 years,68349.0,1475278.0,4632.957314,102969,80
79,81 years,70621.0,1381641.0,5111.385664,102969,81
80,82 years,71124.0,1241341.0,5729.610155,102969,82
81,83 years,73524.0,1151190.0,6386.782373,102969,83


In [171]:
fig = go.Figure()
fig.add_trace(go.Bar(x=usa_total["age"], y=usa_total["Population"]))
fig.add_trace(go.Bar(x=usa_total["age"], y=usa_total["Rate"]))

In [172]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=usa_total["age"], y=usa_total["Rate"], mode="lines"))

In [181]:
df.sort_values(by="age", inplace=True)
df

,State,Cause Of Death,ICD-10 113 Cause List Code,Ten-Year Age Groups,Ten-Year Age Groups Code,Deaths,Population,Crude Rate,Rate,fips,Year,age
2954,Colorado,Other Nontransport Accident,GR113-123,< 1 year,1,33.0,64561.0,51.1,51.11,08,2019,1
4127,Florida,Heart Disease (All),GR113-054,< 1 year,1,22.0,221463.0,9.9,9.93,12,2019,1
8222,Kentucky,#Certain conditions originating in the perinat...,GR113-108,< 1 year,1,120.0,53154.0,225.8,225.76,21,2019,1
18171,Pennsylvania,Accidental Drowning,GR113-120,1-4 years,1-4,13.0,563212.0,Unreliable,2.31,42,2019,1
2386,California,Nontransport accidents,GR113-117,< 1 year,1,56.0,462589.0,12.1,12.11,06,2019,1
...,...,...,...,...,...,...,...,...,...,...,...,...
6050,Illinois,Heart Disease (Major),GR113-053,85+ years,85+,13602.0,266969.0,5095.0,5094.97,17,2019,85
21094,Vermont,Heart Disease (Major),GR113-053,85+ years,85+,781.0,14437.0,5409.7,5409.71,50,2019,85
5619,Idaho,Heart Disease (Major),GR113-053,85+ years,85+,1627.0,29774.0,5464.5,5464.50,16,2019,85
3516,Delaware,Heart Disease (Major),GR113-053,85+ years,85+,1109.0,20254.0,5475.5,5475.46,10,2019,85


In [199]:
fig = px.scatter(df, x="age", y="Deaths", color="Cause Of Death")
fig.update_coloraxes(showscale=False)
fig.update_traces(showlegend=False)
fig.show()